### FlightAware Web Scraping - by Jérome Blanchet

In [ ]:
#this program is using a Github web scrapping program located here
#https://github.com/Hskl14/Flight-Aware-Web-Scraping

#First, I am customizing the program for Canada Airport only
#Second, I am fixing a few bugs. For example, the original web scrapping program is visiting one airport page 
#at a time at FlightAware.com, and will stop running if the airport page is empty.
#I am solving this by adding a simple 'try except pass' syntax. This is important, because 
#several airports are not active. Next step would be to add time of departure and arrival. 
#and also create an automation, which will run automatically few times a day, and store all arrival and departure 
#in a database.

In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install datapackage

  Created wheel for ijson: filename=ijson-2.6.1-py3-none-any.whl size=17429 sha256=503f0c3fece959c3de83d2bb88cfd7a58b015eefb00036fa6344510e501ea9f7
  Stored in directory: c:\users\18195\appdata\local\pip\cache\wheels\25\89\85\4d495d47fbf7f7f1c8509c4ade3a2f0e65eca07f072d277247
  Created wheel for linear-tsv: filename=linear_tsv-1.1.0-py3-none-any.whl size=7389 sha256=e6743ccad4489257f7e428f783cfdc248c59e65af4e84a6ee32a33894b980c80
  Stored in directory: c:\users\18195\appdata\local\pip\cache\wheels\dd\0a\62\49ee84fab92057fb3e303fb1a59e75ccac6899b417f441305a
Successfully built ijson linear-tsv
  Attempting uninstall: docutils
    Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16
Note: you may need to restart the kernel to use updated packages.


In [74]:
conda install -c conda-forge geckodriver

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\18195\Anaconda3

  added / updated specs:
    - geckodriver


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    geckodriver-0.26.0         |       he025d50_0         1.9 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.1 MB

The following NEW packages will be INSTALLED:

  geckodriver        conda-forge/win-64::geckodriver-0.26.0-he025d50_0
  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m

The following packages will be UPDATED:

  conda                       pkg



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda




In [1]:
#import python modules and packages

from bs4 import BeautifulSoup
from selenium import webdriver
from datapackage import Package
import time
import sys
import argparse
import networkx as nx
import pandas as pd

In [2]:
#getting list of all airport around the world

def get_airport_list():
	print('Getting Airport Data.....')
	package = Package('https://datahub.io/core/airport-codes/datapackage.json')
	for resource in package.resources:
	    if resource.descriptor['datahub']['type'] == 'derived/csv':
	        data = resource.read(keyed=True)
	        print('Got Airport Data.....')
	        return data
	return -1

airport_data = get_airport_list()

Getting Airport Data.....
Got Airport Data.....


In [3]:
#putting in list of tuple all airport located in Canada, and with valid gps code

airport_name = []
airport_gps = []

for airport in airport_data:
    if airport['iso_country'] == 'CA':
        if airport['gps_code'] != None:
            if airport['iata_code'] != None:
                airport_name.append(airport['name']) 
                airport_gps.append(airport['gps_code'])

airport_to_visit_canada = list(zip(airport_name, airport_gps))

In [56]:
airport_to_visit_canada

[('Lady Franklin Point Airport', 'CYUJ'),
 ('Lupin Airport', 'CYWO'),
 ('Terrace Bay Airport', 'CYTJ'),
 ('Bedwell Harbour Seaplane Base', 'CAB3'),
 ('Tofino Harbour Seaplane Base', 'CAB4'),
 ('Alice Arm/Silver City Seaplane Base', 'CAC3'),
 ('Nanaimo Harbour Water Airport', 'CAC8'),
 ('Trail Airport', 'CAD4'),
 ('Merritt Airport', 'CAD5'),
 ('Campbell River Seaplane Base', 'CAE3'),
 ('Whistler/Green Lake Water Aerodrome', 'CAE5'),
 ('Bamfield Seaplane Base', 'CAE9'),
 ('Big Bay Seaplane Base', 'CAF6'),
 ('Chilko Lake (Tsylos Park Lodge) Airport', 'CAG3'),
 ('Pender Harbour Seaplane Base', 'CAG8'),
 ('Ocean Falls Seaplane Base', 'CAH2'),
 ('Courtenay Airpark', 'CAH3'),
 ('Creston Valley Regional Airport - Art Sutcliffe Field', 'CAJ3'),
 ('Anahim Lake Airport', 'CAJ4'),
 ('Douglas Lake Airport', 'CAL3'),
 ('Fort MacKay/Albian Aerodrome', 'CAL4'),
 ('Tahsis Seaplane Base', 'CAL9'),
 ('Duncan Airport', 'CAM3'),
 ('Port Simpson Seaplane Base', 'CAN8'),
 ('Sechelt-Gibsons Airport', 'CAP3'),

In [4]:
len(airport_to_visit_canada)

464

In [5]:
#list of major airport in canada

a = [('Calgary International Airport', 'CYYC'),
('Edmonton International Airport', 'CYEG'),
('Gander International Airport', 'CYQX'),
('Greater Moncton International Airport', 'CYQM'),
('Halifax / Stanfield International Airport', 'CYHZ'),
('John C. Munro Hamilton International Airport', 'CYHM'),
('London Airport', 'CYXU'),
('Montreal / Pierre Elliott Trudeau International Airport', 'CYUL'),
('Ottawa Macdonald-Cartier International Airport', 'CYOW'),
('Quebec Jean Lesage International Airport', 'CYQB'),
('Regina International Airport', 'CYQR'),
('Saskatoon John G. Diefenbaker International Airport', 'CYXE'),
("St. John's International Airport", 'CYYT'),
('Lester B. Pearson International Airport', 'CYYZ'),
('Vancouver International Airport', 'CYVR'),
('Victoria International Airport', 'CYYJ'),
('Winnipeg / James Armstrong Richardson International Airport', 'CYWG')]

In [22]:
def get_flights(i):
    
    try:
        airport_name, airport_gps_code = i[0], i[1]
        print('************* LIST OF DEPARTURES & ARRIVALS FOR ' + airport_name + ' *************')
        driver = webdriver.Firefox()
        driver.get('https://tr.flightaware.com/live/airport/' + airport_gps_code);
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        departures = soup.find(id='departures-board')
        arrivals = soup.find(id='arrivals-board')
        departures_table_body = departures.find('tbody')
        arrivals_table_body = arrivals.find('tbody')

        for tr in arrivals_table_body.find_all('tr'):
            flight_destination = tr.find_all('td')[2].text.strip()
            if flight_destination is "":
                continue
            edges.append((flight_destination[:-5], airport_name))
            print("From: " + flight_destination[:-5])
            if flight_destination[-5] is not '(':
                continue

        for tr in departures_table_body.find_all('tr'):
            flight_destination = tr.find_all('td')[2].text.strip()
            if flight_destination is "":
                continue
            edges.append((airport_name ,flight_destination[:-5]))
            print("To: " + flight_destination[:-5])
            if flight_destination[-5] is not '(':
                continue
        driver.quit()
        
    except AttributeError as error:
        pass

    return

In [23]:
starting_time = time.time()

edges = list()

for i in a:
    get_flights(i)

print('************* ELAPSED TIME: ' + str(time.time()-starting_time) + ' *************')

************* GETTING FLIGHTS FOR Calgary International Airport *************
From: Regina Int'l 
From: Winnipeg Int'l 
From: Edmonton Int'l 
From: Cincinnati/Northern Kentucky International Airport 
From: Joe Foss Field 
From: Memphis Intl 
From: Edmonton Int'l (
From: Winnipeg Int'l 
From: Hamilton/John C. Munro Int'l 
From: Toronto Pearson Int'l (
From: Vancouver Int'l (
From: Lethbridge County (Lethbridge) (
From: Lethbridge County (Lethbridge) (
From: Toronto Pearson Int'l 
From: Vancouver Int'l 
From: Edmonton Int'l 
From: Halifax Int'l 
From: Fort MacKay/Firebag Aerodrome 
From: Regina Int'l 
From: Vancouver Int'l 
To: Edmonton Int'l 
To: Edmonton Int'l (
To: Vancouver Int'l 
To: Victoria Int'l 
To: Toronto Pearson Int'l 
To: Fort McMurray 
To: Toronto Pearson Int'l 
To: Regina Int'l 
To: Edmonton Int'l 
To: Vancouver Int'l 
To: Fort MacKay/Firebag Aerodrome 
To: Fort MacKay/Horizon 
To: Hamilton/John C. Munro Int'l 
To: Vancouver Int'l (
To: Victoria Int'l (
To: Winnipeg Int'l 